## 3. Feature_engineering_and_algorithm_setup

Authors : Haddam Yacine, Ka Alioune, Renaud Adrien

<p align="center">
  <a>
    <img src="../src/figures/logo-hi-paris-retina.png" alt="Logo" width="280" height="180">
  </a>

  <h3 align="center">Data Science Bootcamp</h3>
</p>

In this lab, we briefly recall some machine learning basics, and we are interested in a problem of building a regression model using machine learning algorithms.

## What is a machine learning model:

The Building a machine learning model can be summed up in finding a link function $f$
 ($Y=f(X) + \epsilon$) which is most often the
result of error minimization : <p style="text-align: center;">$\sum_i E(Y_i,f(X_i))$</p> where
$(X_i,Y_i)$ is a list of pairs (features, target).

**Objective:** 
- Train the model from a dataset and assess its ability to generalize on unseen data
- Understand the explanatory factors of our target
    
**Method:**
- Separate the target variable from the features
- separate the data into three samples (train / validation / test)
- train the model (on the train set) and evaluate its performance (on the test set).

## Instructions for this tutorial:
- Load the dataset from workspace.
- Train a regression tree to predict the `meter_reading`, plot the importance of each of the features of the database. Evaluate with metrics on train and test.
- Train multiple regression trees by varying the max_depth parameter. Evaluate performance with metrics, on train and test, and plot the curve of these metrics as a function of max_depth.
- Train multiple random forests by varying some parameters. Evaluate performance with metrics, on train and test, and plot the curve of these metrics as a function of complexity.

We will use the [sklearn package](https://fr.wikipedia.org/wiki/Scikit-learn) for models and various metrics

### Data Path

`data_dir` is the path to data folder.

In [ ]:
data_dir = "/home/jovyan/personal_workspace/bootcamp/data"

## Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

sys.path.append('../src/notebooks')
from utils.get_data import load_data
from utils.utils_model import plot_importance, plot_max_depth_influence, plot_n_estimators_influence


pd.set_option('display.max_columns', 500)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

## Load data

In [ ]:
data = pd.read_feather(os.path.join(data_dir, "merged/data.feather"))

# remove building 1099 that has abnormal behaviour
data = data[data.building_id != 1099]

## Features of the Dataset

- Building metadata
   * `building_id`: unique identifier of the building.
   * `site_id`: unique identifier of the site.
   * `primaryspaceusage`: Primary space usage of all buildings is mapped using the [energystar scheme building description types](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/identify-your-property-type). 
   * `square_feet`: Floor area of building in square meters (m2).
   *  `lat`: Latitude of building location to site level.
   *  `lng`: Longitude of building location to site level.


- Weather data
   * <code>timestamp</code>: date and time in the format YYYY-MM-DD hh:mm:ss. Local timezone.
   * <code>site_id</code>: unique identifier of the site.
   * <code>air_temperature</code>: The temperature of the air in degrees Celsius (ºC).
   * <code>cloud_coverage</code>: Portion of the sky covered in clouds, in [oktas](https://en.wikipedia.org/wiki/Okta).
   * <code>dew_temperature</code>: The dew point (the temperature to which a given parcel of air must be cooled at constant pressure and water vapor content in order for saturation to occur) in degrees Celsius (ºC).
   * <code>precip_depth_1_hr</code>: The depth of liquid precipitation that is measured over a one hour accumulation period (mm).
   * <code>sea_lvl_pressure</code>: The air pressure relative to Mean Sea Level (MSL) (mbar or hPa).
   * <code>wind_direction</code>: The angle, measured in a clockwise direction, between true north and the direction from which the wind is blowing (degrees).
   * <code>wind_speed</code>: The rate of horizontal travel of air past a fixed point (m/s).


- Meter reading data
    *   `timestamp`: date and time in the format YYYY-MM-DD hh:mm:ss. 2016 and 2017 data.
    *   `building_id`: unique identifier of the building.
    *   `meter_reading`: meter reading in kilowatt hour (kWh) .
    *   `meter`: meter type, `chilledwater`, `electricity`, `hotwater` or `steam` .

```json
{0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}
```

It is always a good practice to explicitly state the features that we would like to use before training machine learning algorithms.

In [ ]:
columns_to_use = [
    'site_id', 'building_id',
    'timestamp',
    'lat', 'lng',
    'primary_use',
    'square_feet',
    'air_temperature', 'dew_temperature', 'precip_depth_1_hr', 'wind_speed', 'sea_level_pressure', 'wind_direction',
    'meter_name',
    'meter_reading'
]
data = data[columns_to_use]

## Feature Engineering

Feature engineering is the process by which knowledge of data is used to construct explanatory variables, features, that can be used to train a predictive model. Engineering and selecting the correct features for a model will not only significantly improve its predictive power, but will also offer the flexibility to use less complex models that are faster to run and more easily understood.

Based on [EDA of meter readings]():

*   _Healthcare_,  and _Education_ usages shows the highest meter reading values while _Parking_ shows lower.
*   _Hotwater_ meter shows the highest meter reading values.
*   Monthly behaviour (meter-reading median) shows higher readings in warm season.
*   Hourly behaviour (meter-reading median) shows higher values from 6 to 19 hs.
*   Weekday behaviour: lowers during weekends.
* There is also a significant difference between sites

Based on these conclusions, we can create new features to in order to have an accurate model or to avoid complex one

### Feature Engineering from timestamps

The timestamp in itself is not a useful feature. But we can extract from it some powerful features like the hour of the day or the day of the week.

In [ ]:
# hour
data["hour"] = data["timestamp"].dt.hour.astype("int8")
# days of the week (mon=0 and sun=6)
data["weekday"] = data["timestamp"].dt.dayofweek.astype("int8")
# month
data["month"] = data["timestamp"].dt.month.astype("int8")
# year
data["year"] = data["timestamp"].dt.year.astype("int16")
# days (1 to 31)
data["day"] = data["timestamp"].dt.day.astype("int8")

Do you think that there is some other features that could be interesting?

```python
# business hours
data['is_wider_busness_hours'] = np.where((data["hour"] >= 7) & (data["hour"] <=19 ), 1, 0)

# Weekend
data['is_weekend'] = np.where((data["weekday"] >= 0) & (data["weekday"] <= 4), 0, 1)

# Season of year
data['season'] = (np.where(data["month"].isin([12, 1, 2]), 0,
                   np.where(data["month"].isin([3, 4, 5]), 2,         
                   np.where(data["month"].isin([6, 7, 8]), 3,          
                   np.where(data["month"].isin([9, 10, 11]), 1, 0)))))
```

### Feature Engineering from geographic position

We have seen a difference between US and European sites. Lets create a feature for this.

In [ ]:
data["zone_geo"] = "US"
data.loc[data.lng > -4, 'zone_geo'] = "EUROPE"

### Feature Engineering  from transformation 

Feature transformations can include aggregating, combining transforming attributes to create new features. Useful and relevant features will depend on the problem at hand but averages, sums, log or ratios can better expose trends to a model.

We can also transform a numerical feature into a categorical feature by cutting it into classes. This can be interesting to avoid the impact of outliers or to reduce the variance of the output variable.

##### Example: 

```python
# log transformation
data['square_feet_log'] = data['square_feet'].apply(np.log)

# polynomial transformation
data['air_temperature_squared'] = data['air_temperature']**2
```

## Encoding your data

Some algorithms can work with categorical data directly. This means that categorical data must be converted to a numerical form. 
To Convert Categorical Data to Numerical Data, this involves two steps :

- Integer  (ordinal or cardinal)
- One-Hot Encoding

<img src="../src/figures/encoding.png" width=900 height=500 />

Use [pd.get_dummies()](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) for OneHotEncoding

In [ ]:
# list of columns to encode using One-Hot-Encoding
columns_to_encode = ["meter_name", "primary_use", "zone_geo", "site_id"]

# encode those columns
encoded_data = pd.get_dummies(data[columns_to_encode], columns=columns_to_encode)

# add encoded columns to the data
data = pd.concat([data, encoded_data], axis=1)

In [ ]:
data.head()

## Train / Validation / Test Split 

The **train-validation-test** split is a technique for training and evaluating the performance of a machine learning algorithm.

The **procedure** involves taking a dataset and dividing it into three subsets:
- The **train** subset is used to fit the model and is referred to as the training dataset. You should not evaluate the performance of the model on this train set,
- The **validation** subset is used to tune hyperparameters of an algorithm. For example the *max_depth* for a regression tree,
- The **test** subset is not used to train the model; it is only used at the end to evaluate the performances of the model. For a purist, once you 'opened' the test set, you should not modify the model anymore.


**Strategy**
   - Define a test set, and separate the remaining between train and validation.
   - Generally they represent respectively 70% | 15% | 15% of the initial dataset.

<img src="../src/figures/train_test_1.png" width=700 height=400 />

In [ ]:
train = data[data["timestamp"].between("2016-01-01 00:00:00", "2016-12-31 23:00:00")].reset_index(drop=True)
val = data[data["timestamp"].between("2017-01-01 00:00:00", "2017-09-01 00:00:00")].reset_index(drop=True)
test = data[data["timestamp"] > "2017-09-01 00:00:00"].reset_index(drop=True)

In [ ]:
print(f"Number of examples in the train set: {train.shape[0]}")
print(f"Number of examples in the val set:   {val.shape[0]}")
print(f"Number of examples in the test set:  {test.shape[0]}")

</h3><font color='red'>  ! Be sure that all buildings in the test and validation sets are also in train set !</font></h3>

In [ ]:
# List of unique buildings in tain set
train_building = train.building_id.unique().tolist()

# Keep only buildings that are in train set
train = train[train.building_id.isin(train_building)].reset_index(drop=True)
test = test[test.building_id.isin(train_building)].reset_index(drop=True)
val = val[val.building_id.isin(train_building)].reset_index(drop=True)

## Splitting your data (Features / Target)

It is a rather practical approach because generally the algorithms of machine learning ask for the features on the one hand and the target on the other hand.

<img src="../src/figures/split_columns.png" width=700 height=500 />

In [ ]:
features = [
    # 'building_id',
    'lat',
    'lng',
    'square_feet',
    'air_temperature',
    'dew_temperature',
    'precip_depth_1_hr',
    'wind_speed',
    'sea_level_pressure',
    'wind_direction',
    'hour',
    'weekday',
    'month',
    'meter_name_chilledwater',
    'meter_name_electricity',
    'meter_name_hotwater',
    'meter_name_steam',
    'primary_use_Education',
    'primary_use_Entertainment/public assembly',
    'primary_use_Healthcare',
    'primary_use_Industry',
    'primary_use_Lodging/residential',
    'primary_use_Office',
    'primary_use_Other',
    'primary_use_Parking',
    'primary_use_Public services',
    'primary_use_Services',
    'zone_geo_EUROPE',
    'zone_geo_US',
    'site_id_0',
    'site_id_1',
    'site_id_2',
    'site_id_3',
    'site_id_4',
    'site_id_5',
    'site_id_6',
    'site_id_7',
    'site_id_9',
    'site_id_11',
    'site_id_12',
    'site_id_13',
    'site_id_15',
]

target = "meter_reading"

In [ ]:
print(f"::: Number of features {len(features)}")

In [ ]:
# save datasets for later use

SAVED_DATA_MODEL = True

PATH_MODEL_DATA = os.path.join(data_dir, "model")
if not(os.path.exists(PATH_MODEL_DATA)):
    os.mkdir(PATH_MODEL_DATA)

if SAVED_DATA_MODEL:
    train[features + [target]].to_feather(os.path.join(PATH_MODEL_DATA, 'train.feather'))
    val[features + [target]].to_feather(os.path.join(PATH_MODEL_DATA, 'val.feather'))
    test[features + [target]].to_feather(os.path.join(PATH_MODEL_DATA, 'test.feather'))

## Evaluation Metrics for a Regression Problem

 In this lab, as we are interested in a regression problem, we will first see some classic regression metrics.

Suppose we evaluate these metrics for a set $(y_i, \hat{y}_i)_{i=1,...n_{\text{test}}}$, where $y_i$ is the true value and   $\hat{y_i}$ is the prediction.


- **mean absolute error**:   
$\text{MAE} = \frac{1}{n_{\text{test}}} \sum_{i=1,...n_{\text{test}}} |y_i - \hat{y_i}|$ .

- **mean squared error**:   
$\text{MSE} = \frac{1}{n_{\text{test}}} \sum_{i=1,...n_{\text{test}}} (y_i - \hat{y_i})^2$. The most used.

- **max error**: 
$\text{MAX_Error} =  \max_{i=1,...n_{\text{test}}} (y_i - \hat{y_i})$. Calculates the maximum residual error. It is very sensitive to outliers

MSE has the benefit of penalizing large errors more so can be more appropriate in some cases, for example, if being off by 10 is more than twice as bad as being off by 5.

From an interpretation standpoint, MAE is clearly the winner. MSE does not describe average error alone and has other implications that are more difficult to tease out and understand.

<img src="../src/figures/metric.PNG" width=700 height=400 />

## Regression trees

In this lab, we are interested in regression trees and random forests, for our regression problem. They are among the simplest models in machine learning, but remain important because they are one of the few models that remain explainable: it is the opposite of the black box model.

A regression tree is a set of ordered decision rules, as in the figure below. There are many types of decision trees, but in machine learning they often take the form of binary trees. By going through the tree, we go through a succession of questions on the variables of the database, until ending up on a sheet, which allows us to know which output to assign to a given input.

<img src="../src/figures/arbre_1.png" width=700 height=400 />

At this stage, we can ask the following question: a given tree is easy to read, but on what bases, on what rules was it built? A classic tree-building algorithm is the CART (Classification and Regression Trees) algorithm.

To formalize a little, we can say that in this algorithm, we suppose that the tree divides the input space into M regions $ R1, ..., RM $. The value assigned in output for an input x can be represented in the form of a decision function f such as:

<p style = "text-align: center;"> $f (x) = \sum_{m = 1, ..., M} c_m \times \mathbb {I} \left (x \in R_m \right) $ </p>
 
Where the $ c_m $ are constants to be determined during training.

$\textbf {When do we stop splitting our regions?}$ The depth of the tree is generally considered to be a hyperparameter to be optimized. The classical algorithms therefore split in a binary way until the size of the tree reaches this parameter (which will be called in the following
$ max \_depth $), then then do some pruning. Pruning consists of removing certain nodes from the tree, based on a cost-complexity function. [see here](https://towardsdatascience.com/decision-tree-classifier-and-cost-computation-pruning-using-python-b93a0985ea77).


**PROS**

- direct translation of the tree into a rule base
- undifferentiated treatment of different types of predictor variables
- robust against outliers, solutions for missing data
- speed and ability to handle very large databases

**CONS**

- stability problem
- poorer performance in general compared to other methods

### Fit

We fit a decision tree on the train set.

In [ ]:
from sklearn import tree


model_tree = tree.DecisionTreeRegressor(max_depth=3, random_state=0)
model_tree = model_tree.fit(train[features], train[target])

We can visualize the fitted decision tree.

In [ ]:
fig = plt.figure(figsize=(25, 20))
_ = tree.plot_tree(model_tree,
                   feature_names=train[features].columns,
                   filled=True)

### Evaluation

Evaluate model performance on the validation set.  
For this we compute the MAE, MSE and maximum error.

In [ ]:
from sklearn.metrics import (
    mean_squared_error,
    max_error,
    mean_absolute_error,
    mean_squared_log_error,
)


def print_errors(model, X, y):
    y_pred = model.predict(X)

    print(f'MAE : {mean_absolute_error(y, y_pred):.0f}')
    print(f'MSE : {mean_squared_error(y, y_pred):.0f}')
    print(f'MAX : {max_error(y, y_pred):.0f}')

In [ ]:
print_errors(model_tree, val[features], val[target])

### Feature importance

Feature importance refers to a class of techniques for assigning scores to input features to a predictive model that indicates the relative importance of each feature when making a prediction.

The scores are useful and can be used in a range of situations in a predictive modeling problem, such as:

- Better understanding a model.
- Reducing the number of input features.

In [ ]:
plot_importance(model_tree, train[features])

## [Bias / Variance Trade-off](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229)

<img src="../src/figures/overfiting.PNG" width=600 height=200 />

$\textbf{UNDERFITTING :}$ The model is too simple to capture the relationships between the data

*Solutions*:
- Introduce more features
- Increase model complexity


$\textbf{OVERFITTING :}$ The model is too complex and sticks too closely to the training data

*Solutions*:

- Decrease model complexity
- Include more data
- Use regularization

In [ ]:
max_depth_ls = [2, 4, 8, 16, 32, 48]
plot_max_depth_influence(
    max_depth_ls,
    train[features], train[target],
    val[features], val[target],
)

## Random Forests

The main problem with decision trees is their large variance: a tiny error at the top of the tree is propagated all the way down the tree and it gets worse quickly. To stabilize the tree's predictions, we prefer to generate a set of trees, a forest and this algorithm is called *Random Forest*.


To create a random forest with B trees, we proceed as follows:

- For i ranging from 1 to B:
  - We draw randomly with replacement a sub-sample of the data size $ n <n _ {\ text {train}} $
  - We randomly draw a subsample of features of size m with in general $ m \leq \sqrt {p} $
  - On this new dataset composed of n examples and m features, we train a decision tree of fixed max depth
- We thus obtain $ B $ decision trees. If we denote by $ f1, ..., fB $ the prediction functions of each tree, then in regression, the decision function of the forest $ f_ {RF} $ will be:
<p style = "text-align: center;"> $ f_{RF} (x) = \frac{1}{B} \sum_{i = 1, ..., B} f_i (x)$ </p>

<img src="../src/figures/RF.png" width=900 height=500 />

### The mains parameters

- *n_estimators* : number of trees in the foreset

- *max_features* : max number of features considered for splitting a node

- *max_depth* : max number of levels in each decision tree

### Fit 

In [ ]:
from sklearn.ensemble import RandomForestRegressor


model_rf = RandomForestRegressor(
    n_estimators=10,
    max_depth=16,
    random_state=0,
    n_jobs=-1
)
model_rf.fit(train[features], train[target])

### Evalution

Evaluate model performance on the validation set.  
For this we compute the MAE, MSE and maximum error.

In [ ]:
print_errors(model_rf, val[features], val[target])

### Plot importance

In [ ]:
plot_importance(model_rf, train[features])

## To Do

<font color='blue'> <h2> Feature Engeneering </h2> </font>

1. Based of the feature importance of the Regression tree, drop some insignificant features on *train* and fit a new model. Is your model better now ?

2. Using your background, create a new feature who seems interesting (do it on train, val and test). What is the impact of this feature on your model ?


**Hint 1**

```python
selected_features = [...]

model_rf = RandomForestRegressor(
    n_estimators=10,
    max_depth=16,
    random_state=0,
    n_jobs=-1
)
model_rf.fit(train[selected_features], train[target])
print_errors(model_rf, val[selected_features], val[target])
```

**Hint 2**

```python
# Weekend
train['is_weekend'] = np.where((train["weekday"] >= 0) & (train["weekday"] <= 4), 0, 1)
val['is_weekend'] = np.where((val["weekday"] >= 0) & (val["weekday"] <= 4), 0, 1)

model_rf = RandomForestRegressor(
    n_estimators=10,
    max_depth=16,
    random_state=0,
    n_jobs=-1
)
model_rf.fit(train[features + ["is_weekend"]], train[target])
print_errors(model_rf, val[features + ["is_weekend"]], val[target])
````


<font color='blue'> <h2> Bias / Variance Trade-off with hyperparameters </h2> </font> 

1. Using the validation sample, vary the parameter `n_estimators` of the random forest from 1 to 20 in steps of ~5.

2. Analyze the results and give the optimal number of tree.

**Hint**: use the `plot_n_estimators_influence()` from `utils_model.py` which takes an integer list as argument:
```python
n_estimators = [1, 2, 5, 10, 15, 30]
plot_n_estimators_influence(n_estimators, X_train, y_train, X_val, y_val)
```

<font color='blue'> <h2> Analyze deeper your model performances </h2> </font> 

1. Add the predictions of your model to the `val` dataframe
2. For each lines compute the absolute error
3. Compute also the absolute percentage error
4. What can you say about your model predictions? Are they as good for every building, day...


**Hint 1 - 2 - 3**

```python
val["meter_reading_prediction"] = model_rf.predict(val[features])
val["MAE"] = (val["meter_reading"] - val["meter_reading_prediction"]).abs()
val["MAPE"] = (val["meter_reading"] - val["meter_reading_prediction"]).abs() / val["meter_reading"] * 100
```


**Hint 4**

```python
val[val.meter_reading > 0.1].groupby("building_id").MAPE.median().sort_values()

(
    val[val.meter_reading > 0.1]
    .set_index("timestamp")
    .resample("1D")
    .MAPE
    .median()
    .plot.line(figsize=(20, 10))
)
plt.grid()


val[val.meter_reading > 0.1].plot.scatter("meter_reading", "meter_reading_prediction", alpha=0.1, figsize=(10, 10))
plt.xlim(0, 10000)
plt.ylim(0, 10000)
```

## Go further!
- Hastie T., Tibshirani R., Friedman J., « [The elements of Statistical Learning](http://statweb.stanford.edu/~tibs/ElemStatLearn/) - Data Mining, Inference and Prediction », pringer, 2009.